In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm

In [3]:
#List of file to collect
mypath = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2009/" #'data/2009/2009/'
file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
len(file_names)

290832

In [3]:
#Year 2015 also because we analyse 2016 and 2015 for 2014
expect_classes_G06F_2014, expect_classes_H01L_2014, expect_classes_A61B_2014 = [], [], []
with open(r'data/expectation_space/2014_G06F_expectation_IPC_class.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        expect_classes_G06F_2014.append(x)

with open(r'data/expectation_space/2014_H01L_expectation_IPC_class.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        expect_classes_H01L_2014.append(x)
        
with open(r'data/expectation_space/2014_A61B_expectation_IPC_class.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        expect_classes_A61B_2014.append(x)

In [4]:
#Getting files in this XXXX year related to main classes and other classes
knowledge_space_G06F = []
knowledge_space_H01L = []
knowledge_space_A61B = []

expectation_space_G06F_2014 = []
expectation_space_H01L_2014 = []
expectation_space_A61B_2014 = []

for i in tqdm(range(len(file_names))):
    patent_path = mypath + file_names[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    
    class_mainIPC = d['main_ipcr_label']
    class_main = class_mainIPC[0:4]
    #We are collecting all documents related to the main class - we distinguish them later by date
    if class_main == 'G06F':
        knowledge_space_G06F.append(file_names[i])
    if class_main == 'H01L':
        knowledge_space_H01L.append(file_names[i])
    if class_main == 'A61B':
        knowledge_space_A61B.append(file_names[i])
    
    #For the expectations states - we have one for each year since the class are not similar
    #2014
    if class_mainIPC in expect_classes_G06F_2014:
        expectation_space_G06F_2014.append(file_names[i])
    if class_mainIPC in expect_classes_H01L_2014:
        expectation_space_H01L_2014.append(file_names[i])
    if class_mainIPC in expect_classes_A61B_2014:
        expectation_space_A61B_2014.append(file_names[i])

100%|█████████████████████████████████████████████████████████████████████████| 290832/290832 [20:26<00:00, 237.03it/s]


In [11]:
#Saving knowledge space per year in csv
patent_number, decisions, titles, backgrounds, claims = [], [], [], [], []
current_date = 20140101
for i in tqdm(range(len(knowledge_space_A61B))):
    patent_path = mypath + knowledge_space_A61B[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    #Not taking patnts that are not published yet
    if int(d['date_published']) < current_date:
        
        #Creating the lists for the other information
        patent_number.append(d['application_number'])
        titles.append(d['title'])
        backgrounds.append(d['background'])
        claims.append(d['claims'])
        decisions.append(d['decision'])
    else:
        #If the date is superior to 2016, we still take accepted or rejected into account
        if d['decision'] == 'ACCEPTED' or d['decision'] == 'REJECTED':
            #Creating the lists for the other information
            patent_number.append(d['application_number'])
            titles.append(d['title'])
            backgrounds.append(d['background'])
            claims.append(d['claims'])
            decisions.append(d['decision'])

100%|█████████████████████████████████████████████████████████████████████████████| 7754/7754 [00:26<00:00, 287.54it/s]


In [12]:
df_known = pd.DataFrame({'application_number': patent_number, 'title': titles, 'background': backgrounds, 'claims':claims, 'decision': decisions})
print(len(df_known))

7754


In [13]:
df_known.to_csv('data/knowledge_space/20142009_A61B_knowledge_space_raw.csv', index=False)

SyntaxError: invalid syntax (1642478469.py, line 2)

In [20]:
#Saving expectations space per year in csv
patent_number, decisions, titles, backgrounds, claims = [], [], [], [], []
current_date = 20140101
for i in tqdm(range(len(expectation_space_A61B_2014))):
    patent_path = mypath + expectation_space_A61B_2014[i]
    with open(patent_path) as f:
        d = json.load(f)
        f.close()
    
    #Not taking patnts that are not published yet
    if int(d['date_published']) < current_date:
        
        #Creating the lists for the other information
        patent_number.append(d['application_number'])
        titles.append(d['title'])
        backgrounds.append(d['background'])
        claims.append(d['claims'])
        decisions.append(d['decision'])
    else:
        #If the date is superior to 2016, we still take accepted or rejected into account
        if d['decision'] == 'ACCEPTED' or d['decision'] == 'REJECTED':
            #Creating the lists for the other information
            patent_number.append(d['application_number'])
            titles.append(d['title'])
            backgrounds.append(d['background'])
            claims.append(d['claims'])
            decisions.append(d['decision'])

100%|███████████████████████████████████████████████████████████████████████████| 46575/46575 [01:34<00:00, 492.89it/s]


In [21]:
df_expect = pd.DataFrame({'application_number': patent_number, 'title': titles, 'background': backgrounds, 'claims':claims, 'decision': decisions})
print(len(df_expect))

46573


In [22]:
df_expect.to_csv('data/expectation_space/20142009_A61B_expectation_space_raw.csv', index=False)